In [ ]:
#Ladda paket: xlrd behövs för Excel 

from minio import Minio
from minio.commonconfig import Tags
from io import BytesIO
import os
import pandas as pd

import tempfile

try:
    import xlrd
except:
    !pip install xlrd
    import xlrd

try:
    import openpyxl
except:
    !pip install openpyxl
    import openpyxl

try:
    import pyreadstat
except:
    !pip install pyreadstat
    import pyreadstat

#Installera paket för SAS

!pip install sas7bdat
!pip install xport

# function to write object and tags

def read_from_s3(bucket, filepath, datatype='csv'):
    """
    Read a file from an S3 bucket.

    :param bucket: The S3 bucket name.
    :param filepath: The path to the file within the bucket.
    :datatype: 'csv','excel',or 'sas'
    :return: Pandas dataframe  or None if the file does not exist.
    """
    # Create Minio client
    client = Minio(
        os.getenv("JUPYTERLAB_S3_ENDPOINT").replace("http://", ""),
        secure=False,
        access_key=os.getenv("JUPYTERLAB_S3_ACCESS_KEY_ID"),
        secret_key=os.getenv("JUPYTERLAB_S3_SECRET_ACCESS_KEY"),
    )

    # Check if the bucket exists
    found = client.bucket_exists(bucket)
    if not found:
        print("Bucket does not exist: %s" % (bucket))
        return None  # Failure

    # Check if the object exists
    try:
        obj = client.get_object(bucket, filepath)
        # Read the object into a bytes buffer
        buffer = BytesIO(obj.read())
        # Read the bytes buffer into a pandas dataframe
        if datatype=='csv':
            df = pd.read_csv(buffer)
            return df
        elif datatype == 'excel':
            df = pd.read_excel(buffer)
            return df
        elif datatype == 'sas':
            # create a temporary file
            with tempfile.NamedTemporaryFile(mode="wb", delete=False) as tmp:
                # write the bytes data to the file
                tmp.write(buffer.getvalue())
                # get the file name
                tmp_name = tmp.name
            df, metadata = pyreadstat.read_sas7bdat(tmp_name)
            return df
        else:
            print("Only csv, excel, or sas are accepted datatype. Please check your datatype.")
            return None
    except Exception as e:
        print(f"Error reading file {filepath} from bucket {bucket}: {str(e)}")
        return None  # Failure

def write_to_s3(bucket, filepath, tags={}, meta={}):
    # create Minio client and init bucket if necessary
    client = Minio(
        os.getenv("JUPYTERLAB_S3_ENDPOINT").replace("http://", ""),
        secure=False,
        access_key=os.getenv("JUPYTERLAB_S3_ACCESS_KEY_ID"),
        secret_key=os.getenv("JUPYTERLAB_S3_SECRET_ACCESS_KEY"),
    )
    found = client.bucket_exists(bucket)
    if not found:
        print("Bucket does not exist: %s" % (bucket))
        return False  # failure

    # init tags
    t = Tags(for_object=True)
    for k, v in tags.items():
        t[k] = v

    with open(filepath, mode="rb") as source_file:
        contents = source_file.read()

    client.put_object(
        bucket,
        filepath,
        data=BytesIO(contents),
        length=len(contents),
        tags=t,
        metadata=meta,
        content_type='application/csv'
    )
    return True  # success


'''
# Ett exempel med SAS data
# OBS: bucket_name måste justeras till "din" Minio bucket. och file_path_in_s3 också.

bucket_name = "produktion-test"
file_path_in_s3 = "index_orig.sas7bdat"

df = read_from_s3(bucket_name, file_path_in_s3, datatype="sas")
print("Data från a SAS dataset")
print(df.tail())

# Exempel med Excel fil
file2 = "PrisIndex.xlsx"
df2 = read_from_s3(bucket_name, file2, datatype="excel")
print("Data från an Excel fil")
print(df2.tail())

'''


